In [5]:
import pandas as pd

# Load your original CSV file
df = pd.read_csv('../datasets/tripdata_05_2024.csv')

# Convert the 'Start Time' column to datetime format
df['Start Time'] = pd.to_datetime(df['Start Time'])

# Filter for a specific date, for example: May 5, 2024
target_date = '05/05/2024'
filtered_df = df[df['Start Time'].dt.date == pd.to_datetime(target_date).date()]

# Save the filtered data to a new CSV file
filtered_df.to_csv('trips_05_05.csv', index=False)

# Optional: Display the first few rows of the filtered data
filtered_df.head()


ModuleNotFoundError: No module named 'pandas'